# Fine Tuning DistilBERT for MultiLabel Vulnerability Classification Using Opcode

## Prepare 

In [ ]:
!pip install transformers

In [ ]:
!pip install evaluate

In [1]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
# import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [3]:
import evaluate


In [4]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/')

In [6]:
tokenizer

DistilBertTokenizer(name_or_path='/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer_opcode/', vocab_size=52000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [7]:
distil_model = DistilBertModel.from_pretrained("/home/bombbom/Documents/NLP_in_Detection_System/save_model/pre_train_opcode/checkpoint-50000/")
for prama in distil_model.parameters():
    prama.requires_grad=False

Some weights of the model checkpoint at /home/bombbom/Documents/NLP_in_Detection_System/save_model/pre_train_opcode/checkpoint-50000/ were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
distil_model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(52000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [34]:
data = pd.read_pickle("/home/bombbom/Documents/NLP_in_Detection_System/dataset_example/labeled_SBW_datasets.pkl")
data

,address,source_code,categories,lines
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[154, 141, 103]"
1,0x344005c29af957567f0b40950b425ed018b92170,pragma solidity ^0.4.21;\n\n\ncontract DSMath...,"['arithmetic', 'reentrancy', 'arithmetic', 'ar...","[201, 16, 209, 176, 146, 154, 29]"
2,0x44e320110176c11c93e116f6770f13d96deded43,pragma solidity ^0.4.20;\n\n\ncontract Ownable...,['arithmetic'],"[82, 46]"
3,0x5ecd84482176db90bb741ddc8c2f9ccc290e29ce,pragma solidity ^0.4.8;\ncontract Token{\n ...,"['arithmetic', 'unchecked_low_calls', 'arithme...","[99, 38, 78, 48, 49, 60, 93]"
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,pragma solidity 0.4.15;\n\ncontract RegistryIC...,['Other'],"[80, 28, 63]"
...,...,...,...,...
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,pragma solidity ^0.4.24;\n\n\n\n\n/**\n * @tit...,"['reentrancy', 'arithmetic', 'time_manipulatio...","[258, 264, 245, 253, 287]"
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[200, 218, 128, 102]"
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,pragma solidity ^0.4.23;\n\nlibrary SafeMath {...,"['arithmetic', 'arithmetic', 'front_running']","[124, 20, 205]"
47329,0xe2d4b960d0c639633582cddef57528461e62083d,pragma solidity ^0.4.23;\n\n\n/// @title Multi...,"['arithmetic', 'Other', 'unchecked_low_calls',...","[140, 142, 23, 282, 410, 157, 287, 296, 302, 3..."


In [11]:
# data = data.head()

In [35]:
import re
def get_version(source_code):
    x = re.findall("[0][.][0-9][.][0-9]*[0-9]", source_code)
    if len(x)>0:
        return x[0]
    return "0.4.25"

In [36]:
data['version'] = data.source_code.apply(get_version)

In [37]:
def change(version):
    if version =='0.4.99' :
        return '0.5.0'

In [38]:
data['version'].unique()

array(['0.4.18', '0.4.21', '0.4.20', '0.4.8', '0.4.15', '0.4.17',
       '0.4.24', '0.4.4', '0.4.19', '0.4.16', '0.4.25', '0.4.13',
       '0.4.99', '0.5.0', '0.4.22', '0.4.11', '0.4.23', '0.5.2', '0.5.7',
       '0.5.6', '0.5.4', '0.4.0', '0.5.1', '0.4.2', '0.4.3', '0.4.9',
       '0.5.5', '0.4.7', '0.4.10', '0.5.00', '0.5.8', '0.4.14', '0.4.12',
       '0.7.1', '0.5.9', '0.4.6', '0.1.4', '0.0.1', '0.5.3', '0.3.14',
       '0.4.26', '0.1.0', '0.5.10', '0.2.1', '0.4.1', '0.3.6', '0.3.5',
       '0.9.9', '0.0.2', '0.4.5', '0.2.4', '0.3.1', '0.1.5', '0.1.2',
       '0.3.3', '0.1.3', '0.1.1', '0.3.2', '0.6.0'], dtype=object)

In [39]:
value = [ '0.4.8', '0.4.4',  '0.4.0',  '0.4.2', '0.4.3', '0.4.9', '0.4.7', '0.4.10', '0.5.00', '0.4.6', '0.1.4', '0.0.1',  '0.3.14','0.1.0',  '0.2.1', '0.4.1', '0.3.6', '0.3.5','0.9.9', '0.0.2', '0.4.5', '0.2.4', '0.3.1', '0.1.5', '0.1.2','0.3.3', '0.1.3', '0.1.1', '0.3.2']

def filter_rows_by_values(df, col, values):
       df = data[~data[col].isin(values)]
       return df

In [40]:
data = filter_rows_by_values(data, "version", value)

In [41]:
data.reset_index(drop = "True")

,address,source_code,categories,lines,version
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[154, 141, 103]",0.4.18
1,0x344005c29af957567f0b40950b425ed018b92170,pragma solidity ^0.4.21;\n\n\ncontract DSMath...,"['arithmetic', 'reentrancy', 'arithmetic', 'ar...","[201, 16, 209, 176, 146, 154, 29]",0.4.21
2,0x44e320110176c11c93e116f6770f13d96deded43,pragma solidity ^0.4.20;\n\n\ncontract Ownable...,['arithmetic'],"[82, 46]",0.4.20
3,0xcaf187eb618d2335b4130d784a697be96f4b07b9,pragma solidity 0.4.15;\n\ncontract RegistryIC...,['Other'],"[80, 28, 63]",0.4.15
4,0x2ab58a02c4d26bcf3c6116f62ec5c1a6d4c7a53f,pragma solidity ^0.4.18;\n\n// File: openzeppe...,"['arithmetic', 'front_running', 'Other', 'reen...","[37, 166, 199, 103, 167, 175, 80, 178, 84, 86,...",0.4.18
...,...,...,...,...,...
42829,0xc2396239e59d07bd726e94411413a26152e8c733,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[129, 219, 201]",0.4.18
42830,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,pragma solidity ^0.4.24;\n\n\n\n\n/**\n * @tit...,"['reentrancy', 'arithmetic', 'time_manipulatio...","[258, 264, 245, 253, 287]",0.4.24
42831,0xc73f2474001ad1d6aed615af53631148cf98de6b,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[200, 218, 128, 102]",0.4.18
42832,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,pragma solidity ^0.4.23;\n\nlibrary SafeMath {...,"['arithmetic', 'arithmetic', 'front_running']","[124, 20, 205]",0.4.23


In [42]:
data.version = data.version.apply(change)

In [43]:
import os
import solcx
def compiler(input):
    version = get_version(input)
    token = ""
    
    try: 
        solcx.install_solc(version)
        a = solcx.compile_source(
            input,
            output_values=["opcodes"],
            solc_version=version
        )
        
        for key, value in a.items():
            for k in value:
                # print(value[k])
                # print(len(value[k]))
                # token = value[k].split()
                # print(len(token))
                token = value[k]
    except:
        token=""
    
    return token

In [44]:
data.source_code = data.source_code.apply(compiler)

In [45]:
data

,address,source_code,categories,lines,version
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"['Other', 'arithmetic', 'arithmetic']","[154, 141, 103]",None
1,0x344005c29af957567f0b40950b425ed018b92170,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"['arithmetic', 'reentrancy', 'arithmetic', 'ar...","[201, 16, 209, 176, 146, 154, 29]",None
2,0x44e320110176c11c93e116f6770f13d96deded43,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,['arithmetic'],"[82, 46]",None
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,['Other'],"[80, 28, 63]",None
5,0x2ab58a02c4d26bcf3c6116f62ec5c1a6d4c7a53f,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...,"['arithmetic', 'front_running', 'Other', 'reen...","[37, 166, 199, 103, 167, 175, 80, 178, 84, 86,...",None
...,...,...,...,...,...
47325,0xc2396239e59d07bd726e94411413a26152e8c733,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"['Other', 'arithmetic', 'arithmetic']","[129, 219, 201]",None
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"['reentrancy', 'arithmetic', 'time_manipulatio...","[258, 264, 245, 253, 287]",None
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"['Other', 'arithmetic', 'arithmetic']","[200, 218, 128, 102]",None
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"['arithmetic', 'arithmetic', 'front_running']","[124, 20, 205]",None


In [46]:
def mapping(input):
    # return input
    input = input[1:-1].replace("'", "").replace(" ", "").split(",")
    label = [0,0,0,0,0,0,0,0]
    if 'access_control' in input:
        label[0] = 1
    if 'arithmetic' in input:
        label[1] = 1
    if "denial_service" in input:
        label[2] = 1
    if "front_running" in input:
        label[3] = 1
    if "reentrancy" in input:
        label[4] = 1
    if "time_manipulation" in input:
        label[5] = 1
    if "unchecked_low_calls" in input:
        label[6] = 1
    if "Other" in input:
        label[7] = 1
    
    return list(label)

In [47]:
data.categories = data.categories.apply(mapping)

In [48]:
data

,address,source_code,categories,lines,version
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[154, 141, 103]",None
1,0x344005c29af957567f0b40950b425ed018b92170,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 1, 0, 0, 0]","[201, 16, 209, 176, 146, 154, 29]",None
2,0x44e320110176c11c93e116f6770f13d96deded43,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 0]","[82, 46]",None
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 0, 0, 0, 0, 0, 0, 1]","[80, 28, 63]",None
5,0x2ab58a02c4d26bcf3c6116f62ec5c1a6d4c7a53f,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...,"[0, 1, 0, 1, 1, 0, 1, 1]","[37, 166, 199, 103, 167, 175, 80, 178, 84, 86,...",None
...,...,...,...,...,...
47325,0xc2396239e59d07bd726e94411413a26152e8c733,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[129, 219, 201]",None
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"[0, 1, 0, 1, 1, 1, 1, 0]","[258, 264, 245, 253, 287]",None
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[200, 218, 128, 102]",None
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"[0, 1, 0, 1, 0, 0, 0, 0]","[124, 20, 205]",None


In [49]:
data.drop(columns=['version'])

,address,source_code,categories,lines
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[154, 141, 103]"
1,0x344005c29af957567f0b40950b425ed018b92170,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 1, 0, 0, 0]","[201, 16, 209, 176, 146, 154, 29]"
2,0x44e320110176c11c93e116f6770f13d96deded43,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 0]","[82, 46]"
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 0, 0, 0, 0, 0, 0, 1]","[80, 28, 63]"
5,0x2ab58a02c4d26bcf3c6116f62ec5c1a6d4c7a53f,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...,"[0, 1, 0, 1, 1, 0, 1, 1]","[37, 166, 199, 103, 167, 175, 80, 178, 84, 86,..."
...,...,...,...,...
47325,0xc2396239e59d07bd726e94411413a26152e8c733,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[129, 219, 201]"
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"[0, 1, 0, 1, 1, 1, 1, 0]","[258, 264, 245, 253, 287]"
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[200, 218, 128, 102]"
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"[0, 1, 0, 1, 0, 0, 0, 0]","[124, 20, 205]"


In [50]:
data.to_pickle("dataset_example/address_opcode_vul_lines_SBW.pkl")

In [9]:
data = pd.read_pickle("dataset_example/address_opcode_vul_lines_SBW.pkl")

In [10]:
data = data.rename(columns = {"source_code": "text", "categories": "labels"})

In [11]:
data = pd.DataFrame(data)

In [12]:
data

,address,text,labels,lines,version
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[154, 141, 103]",None
1,0x344005c29af957567f0b40950b425ed018b92170,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 1, 0, 0, 0]","[201, 16, 209, 176, 146, 154, 29]",None
2,0x44e320110176c11c93e116f6770f13d96deded43,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 0]","[82, 46]",None
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 0, 0, 0, 0, 0, 0, 1]","[80, 28, 63]",None
5,0x2ab58a02c4d26bcf3c6116f62ec5c1a6d4c7a53f,PUSH1 0x60 PUSH1 0x40 MSTORE PUSH1 0x40 DUP1 M...,"[0, 1, 0, 1, 1, 0, 1, 1]","[37, 166, 199, 103, 167, 175, 80, 178, 84, 86,...",None
...,...,...,...,...,...
47325,0xc2396239e59d07bd726e94411413a26152e8c733,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[129, 219, 201]",None
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"[0, 1, 0, 1, 1, 1, 1, 0]","[258, 264, 245, 253, 287]",None
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,PUSH1 0x60 PUSH1 0x40 MSTORE CALLVALUE ISZERO ...,"[0, 1, 0, 0, 0, 0, 0, 1]","[200, 218, 128, 102]",None
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3...,"[0, 1, 0, 1, 0, 0, 0, 0]","[124, 20, 205]",None


In [13]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05

In [14]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [15]:
train_size = 0.8
train_data=data.sample(frac=train_size,random_state=20)
test_data=data.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

In [16]:
print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (42834, 5)
TRAIN Dataset: (34267, 5)
TEST Dataset: (8567, 5)


In [17]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# Fine tune MLP

In [25]:
choose_model = "MLP"

In [26]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = distil_model
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 8)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

# Fine tune LSTM

In [ ]:
choose_model = "LSTM"

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = distil_model
        self.lstm1 = torch.nn.LSTM(input_size=768,
                            hidden_size=256,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=False) 
        self.pre_classifier = torch.nn.Linear(256*2, 256)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 8)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        # pooler = hidden_state[:, 0]
        self.lstm1.flatten_parameters()
        output, (hidden, cell) = self.lstm1(hidden_state)
        x = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        pooler = self.pre_classifier(x)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

# Fine tune BiLSTM

In [ ]:
choose_model = "BiLSTM"

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = distil_model
        self.lstm1 = torch.nn.LSTM(input_size=768,
                            hidden_size=256,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True) 
        self.pre_classifier = torch.nn.Linear(256*2, 256)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 8)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        # What is output of output_1
        
        hidden_state = output_1[0]
        # pooler = hidden_state[:, 0]
        self.lstm1.flatten_parameters()
        output, (hidden, cell) = self.lstm1(hidden_state)
        x = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        pooler = self.pre_classifier(x)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

## Training

In [27]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [28]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [63]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
    path_name = "Model_opcode_"+ choose_model + str(epoch) + ".pth"
    torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              }, path_name)

In [64]:
for epoch in range(EPOCHS):
    train(epoch)

1it [00:04,  4.74s/it]

Epoch: 0, Loss:  0.683043360710144


1071it [48:56,  2.74s/it]
1it [00:04,  4.02s/it]

Epoch: 1, Loss:  0.5413535833358765


1071it [50:17,  2.82s/it]
1it [00:04,  4.23s/it]

...

Epoch: 6, Loss:  0.4774535000324249


1071it [1:00:15,  3.38s/it]
1it [00:04,  4.52s/it]

Epoch: 7, Loss:  0.505130410194397


540it [35:39,  3.96s/it]


KeyboardInterrupt: 

In [ ]:
checkpoint = torch.load("/home/bombbom/Documents/Multil_Model_Detection/notebook/save/model_lstm_14.pth")

In [29]:
model.load_state_dict(torch.load(f="/home/bombbom/Documents/NLP_in_Detection_System/Model_opcode_{choose_model}5.pth"))

<All keys matched successfully>

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [30]:
def validation(testing_loader):
    # model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [31]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

268it [24:27,  5.47s/it]


In [32]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [34]:
targets

[[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0,

In [38]:
outputs

[[0.038076307624578476,
  0.6713117957115173,
  0.20210817456245422,
  0.1656932383775711,
  0.4771006107330322,
  0.05121851712465286,
  0.38588863611221313,
  0.49854499101638794],
 [0.08322139084339142,
  0.8591791987419128,
  0.23331719636917114,
  0.15917256474494934,
  0.24349242448806763,
  0.09570149332284927,
  0.25915324687957764,
  0.747382640838623],
 [0.07565445452928543,
  0.8655943274497986,
  0.2628176510334015,
  0.17864879965782166,
  0.22900907695293427,
  0.09036117047071457,
  0.28140684962272644,
  0.7166469097137451],
 [0.03890346363186836,
  0.6990672945976257,
  0.1938924342393875,
  0.15956740081310272,
  0.5042672157287598,
  0.050755735486745834,
  0.4079815447330475,
  0.49639177322387695],
 [0.09104171395301819,
  0.8708988428115845,
  0.22764630615711212,
  0.17848123610019684,
  0.21990525722503662,
  0.09236571192741394,
  0.32581672072410583,
  0.7845436930656433],
 [0.08234523236751556,
  0.8243061304092407,
  0.33277371525764465,
  0.2488223612308502

In [37]:
final_outputs

array([[False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False,  True],
       [False,  True, False, ..., False, False,  True],
       ...,
       [False,  True, False, ..., False, False,  True],
       [False,  True, False, ..., False, False,  True],
       [False,  True, False, ..., False, False,  True]])

In [ ]:
clf_metrics.compute(references=targets, predictions=final_outputs)

In [39]:
y_pred=[]
for sample in  outputs:
  y_pred.append([1 if i>=0.5 else 0 for i in sample ] )
y_pred = np.array(y_pred)
y_pred

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1]])

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(targets, y_pred)

0.16925411462589005

In [41]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(targets, y_pred)

array([[[7874,    0],
        [ 693,    0]],

       [[   0, 1912],
        [   0, 6655]],

       [[6212,    1],
        [2354,    0]],

       [[7076,    0],
        [1491,    0]],

       [[5502,  394],
        [2293,  378]],

       [[7823,    0],
        [ 744,    0]],

       [[6245,    1],
        [2320,    1]],

       [[ 812, 2632],
        [ 645, 4478]]])

In [42]:
from sklearn.metrics import classification_report

label_names = ['access_control', 'arithmetic', 'denial_service', 'front_running','reentrancy', 'time_manipulation', 'unchecked_low_calls', 'Other']

print(classification_report(targets, y_pred,target_names=label_names))

              precision    recall  f1-score   support

     label A       0.00      0.00      0.00       693
     label B       0.78      1.00      0.87      6655
     label C       0.00      0.00      0.00      2354
     label D       0.00      0.00      0.00      1491
     label E       0.49      0.14      0.22      2671
     label F       0.00      0.00      0.00       744
     label G       0.50      0.00      0.00      2321
     label H       0.63      0.87      0.73      5123

   micro avg       0.70      0.52      0.60     22052
   macro avg       0.30      0.25      0.23     22052
weighted avg       0.49      0.52      0.46     22052
 samples avg       0.71      0.57      0.59     22052

